# Initialization

## **To access contents of my drive**


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## **Import libraries/packages**


In [0]:
import tensorflow
import cv2
import os
import pickle

import numpy as np
import skimage as sk
from skimage import transform
from skimage import util

RANDOM_SEED = 4
RANDOM_STATE = np.random.RandomState(4)

## Instantiate VGG16 model pre-trained using Places365 data

In [0]:
# -*- coding: utf-8 -*-
'''VGG16-places365 model for Keras
# Reference:
- [Places: A 10 million Image Database for Scene Recognition](http://places2.csail.mit.edu/PAMI_places.pdf)
'''

from __future__ import division, print_function
import os

import warnings
import numpy as np

from keras import backend as K
from keras.layers import Input
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers.core import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import _obtain_input_shape

from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16_Places365(include_top=True, weights='places',
                    input_tensor=None, input_shape=None,
                    pooling=None,
                    classes=365):
    """Instantiates the VGG16-places365 architecture.
    Optionally loads weights pre-trained
    on Places. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
                 'places' (pre-training on Places),
                 or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`, or invalid input shape
        """
    if not (weights in {'places', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `places` '
                         '(pre-training on Places), '
                         'or the path to the weights file to be loaded.')

    if weights == 'places' and include_top and classes != 365:
        raise ValueError('If using `weights` as places with `include_top`'
                         ' as true, `classes` should be 365')


    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten =include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5, name='drop_fc1')(x)

        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='drop_fc2')(x)
        
        x = Dense(365, activation='softmax', name="predictions")(x)

    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = Model(inputs, x, name='vgg16-places365')

    # load weights
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

    elif weights is not None:
        model.load_weights(weights)

    return model


In [0]:
from keras import optimizers


placesvggmodel = VGG16_Places365(weights='places',include_top=False)
# we don't need this since we are not training the vgg16 model
# placesvggmodel.compile(loss='categorical_crossentropy',optimizer =optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0))
                       #optimizer='adam')

58916864/58909656 [==============================] - 3s 0us/step


# Reading Data

## Instantiate the ImageDataGenerator
For training, it will perform augmentation on the fly in addition to pre-processing the data.

For testing, we only want the data to only be pre-processed.

In [0]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


datagen = ImageDataGenerator(
        rotation_range=25,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest', preprocessing_function=preprocess_input
)

In [0]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

### Instantiate the flow_from_directory object. 
After we feed the generator function to predict_generator(), it will become a DirectoryIterator that contains all information about the loaded file like classes, classes in numerical representation, and the filenames.

train_generator is an iterable object, that browses all sub-directories of /content/drive/My Drive/training/ for images. Augmented samples  are generated on the fly while training and lost after training. You can't read again those augmented images.

In [0]:
# train_generator = datagen.flow_from_directory(
#         '/content/drive/My Drive/training', 
#         target_size=(224, 224),class_mode=None,
#        #color_mode="grayscale",
#         shuffle=False, seed=123)

train_generator = datagen.flow_from_directory(
    directory='/content/drive/My Drive/training',
    batch_size=32,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False,
    seed=4)

Found 1500 images belonging to 15 classes.


In [0]:
# test_generator = test_datagen.flow_from_directory(
#         '/content/drive/My Drive/testing',
#         target_size=(224, 224),
#         class_mode=None,
#         #batch_size=1,
#         #color_mode="grayscale",
#         shuffle=False)

test_generator = datagen.flow_from_directory(
    directory='/content/drive/My Drive/testing',
    batch_size=32,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode=None,
    shuffle=False,
    seed=4)

Found 2985 images belonging to 1 classes.


### Old code

In [0]:
spec = Image.open('/content/drive/My Drive/testing/testing/1234.jpg')

In [0]:
spec=img_to_array(spec)

In [0]:
spec.shape

(256, 256, 1)

In [0]:
def to_rgb5(im):
    im.resize((im.shape[0], im.shape[1], 1))
    return np.repeat(im.astype(np.uint8), 3, 2)

In [0]:
spec =to_rgb5(spec)


In [0]:
print(spec.shape)

(256, 256, 3)


In [0]:
new_array = cv2.resize(spec,(224,224))

In [0]:
new_array.shape

(224, 224, 3)

In [0]:
_0jpg = np.expand_dims(new_array, axis=0)

In [0]:
_0jpg.shape

(1, 224, 224, 3)

In [0]:
from keras.applications.vgg16 import preprocess_input
_0jpg=preprocess_input(_0jpg)

In [0]:
im =  placesvggmodel.predict(_0jpg)

In [0]:
im.shape

(1, 7, 7, 512)

In [0]:
im=im.reshape(1,25088)

In [0]:
im.shape

(1, 25088)

## Extract Features from training set. We get a 1500x7x7x512 array. 

### Extract Features and labels from training set

In [0]:
train_features =  placesvggmodel.predict_generator(train_generator,len(train_generator), use_multiprocessing= True)

In [0]:
train_features = train_features.reshape(-1, train_features.shape[1] * train_features.shape[2] * train_features.shape[3])
train_features.shape

(1500, 25088)

In [0]:
from keras.utils import to_categorical

train_labels = to_categorical(train_generator.classes)
print(train_generator.class_indices)

{'Coast': 0, 'Forest': 1, 'Highway': 2, 'Insidecity': 3, 'Mountain': 4, 'Office': 5, 'OpenCountry': 6, 'Street': 7, 'Suburb': 8, 'TallBuilding': 9, 'bedroom': 10, 'industrial': 11, 'kitchen': 12, 'livingroom': 13, 'store': 14}


### Extract Features from test set

In [0]:
test_features =  placesvggmodel.predict_generator(test_generator,len(test_generator), use_multiprocessing= True)

In [0]:
test_features = test_features.reshape(-1, test_features.shape[1] * test_features.shape[2] * test_features.shape[3])
test_features.shape

(2985, 25088)

# Classification

## Training-Validation split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_features,train_labels,test_size=0.3, random_state=RANDOM_STATE)
print(len(X_train), len(X_valid))

1050 450


### Old code

In [0]:
train_features.shape

(1500, 25088)

In [0]:
test_features.shape

(2985, 7, 7, 512)

In [0]:
from itertools import repeat
import pandas as pd

In [0]:
y_labels=[]
labels = ['Coast','Forest','Highway','Insidecity','Mountain','Office','OpenCountry','Street','Suburb','TallBuilding','bedroom','industrial','kitchen','livingroom','store']

for i in labels:
  
  y_labels.extend(repeat(i,100))

In [0]:
len(y_labels)

1500

In [0]:
train_y=np.asarray(y_labels)
# performing one-hot encoding for the target variable

train_y=pd.get_dummies(train_y)
train_y=np.array(train_y)

In [0]:
train_y.shape

(1500, 15)

## Define a MLP as our classifier

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation


input_dim = X_train.shape[1]

model=Sequential()
model.add(Dense(256, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128,activation='sigmoid'))
model.add(Dropout(0.5))

# model.add(Dense(150,input_dim=500,activation='sigmoid'))
# model.add(Dropout(0.5))

model.add(Dense(units=15))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                1935      
_________________________________________________________________
activation_1 (Activation)    (None, 15)                0         
Total params: 6,457,615
Trainable params: 6,457,615
Non-trainable params: 0
_________________________________________________________________


### Load an existing model (if exist)

In [0]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/training/best_model.h5')

with open('/content/drive/My Drive/training/model_train_history.pkl', 'rb') as file:
    history = pickle.load(file)
    history = history.history
    print('Training acc: {}'.format(history['acc'][-1]))
    print('Training loss: {}'.format(history['loss'][-1]))
    print('Validation acc: {}'.format(history['val_acc'][-1]))
    print('Validation loss: {}'.format(history['val_loss'][-1]))

## Train our MLP classifier

In [0]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_valid, Y_valid))

Train on 1050 samples, validate on 450 samples
Epoch 1/30
1050/1050 [==============================] - 2s 2ms/step - loss: 2.9505 - acc: 0.0943 - val_loss: 2.2094 - val_acc: 0.3578
Epoch 2/30
1050/1050 [==============================] - 1s 495us/step - loss: 2.5275 - acc: 0.1714 - val_loss: 2.0183 - val_acc: 0.4089
Epoch 3/30
1050/1050 [==============================] - 1s 495us/step - loss: 2.3423 - acc: 0.2324 - val_loss: 1.8652 - val_acc: 0.4533
Epoch 4/30
1050/1050 [==============================] - 1s 524us/step - loss: 2.1432 - acc: 0.3238 - val_loss: 1.6624 - val_acc: 0.5578
Epoch 5/30
1050/1050 [==============================] - 1s 611us/step - loss: 2.0485 - acc: 0.3124 - val_loss: 1.5156 - val_acc: 0.5733
Epoch 6/30
1050/1050 [==============================] - 1s 505us/step - loss: 1.8508 - acc: 0.4048 - val_loss: 1.3706 - val_acc: 0.6378
Epoch 7/30
1050/1050 [==============================] - 1s 515us/step - loss: 1.7593 - acc: 0.4514 - val_loss: 1.3666 - val_acc: 0.6156
Epo

### Save the latest model

In [0]:
model.save('/content/drive/My Drive/training/best_model.h5')

with open('/content/drive/My Drive/training/model_train_history.pkl', 'wb') as file:
    pickle.dump(history, file)

### Make predictions based on test feature

In [0]:
test_features = test_features.reshape(-1,25088)

predictions_valid = model.predict(test_features,  verbose=1)

2985/2985 [==============================] - 0s 116us/step


### Get class names for each sample

In [0]:
test_results = []
for i in range(len(predictions_valid)):
  pred_idx = np.argmax(predictions_valid[i])
  for label, label_idx in train_generator.class_indices.items():
    if pred_idx == label_idx:
       test_results.append(label)

        
for i in range(len(test_generator.filenames)):
  print(test_generator.filenames[i], test_results[i])

testing/0.jpg TallBuilding
testing/1.jpg Forest
testing/10.jpg OpenCountry
testing/100.jpg Mountain
testing/1000.jpg store
testing/1001.jpg bedroom
testing/1002.jpg Office
testing/1003.jpg bedroom
testing/1004.jpg Forest
testing/1005.jpg OpenCountry
testing/1006.jpg TallBuilding
testing/1007.jpg Forest
testing/1008.jpg livingroom
testing/1009.jpg Street
testing/101.jpg kitchen
testing/1010.jpg TallBuilding
testing/1011.jpg Mountain
testing/1012.jpg Street
testing/1013.jpg industrial
testing/1014.jpg OpenCountry
testing/1015.jpg OpenCountry
testing/1016.jpg Office
testing/1017.jpg Office
testing/1018.jpg Mountain
testing/1019.jpg Highway
testing/102.jpg Insidecity
testing/1020.jpg Coast
testing/1021.jpg Forest
testing/1022.jpg Street
testing/1023.jpg OpenCountry
testing/1024.jpg OpenCountry
testing/1025.jpg store
testing/1026.jpg kitchen
testing/1027.jpg bedroom
testing/1028.jpg OpenCountry
testing/1029.jpg store
testing/103.jpg Coast
testing/1030.jpg Forest
testing/1031.jpg industrial


In [0]:
# for i in range(1):
#   print(i)
#   l = max(predictions_valid[i])
#   for k in range(len(labels)):

    
#     if l==predictions_valid[i][k]:
#       test_results.append(labels[k])
#       print(k)

In [0]:
test_results

In [0]:
Y_train_cat=[]

In [0]:
for i in range(len(Y_train)):
  l = max(Y_train[i])
  for k in range(len(labels)):
    if l==Y_train[i][k]:
      Y_train_cat.append(labels[k])

In [0]:
len(Y_train_cat)

1050

In [0]:
# debugging
count=0


for i in range (len(Y_train_cat)):
  if test_results[i] == Y_train_cat[i]:
    count+=1

acc = (count/len(Y_train_cat))*100
print(acc)   # gives 100%

### Create a list of test file names

In [0]:
img_index=[]

In [0]:
for i in range(0,2988):
  
  if i not in (1314,2938,2962):
    img_index.append(str(i)+".jpg")
  

In [0]:
len(img_index)

2985

### Ceate a list of results in the desired format

In [0]:
run_3 = []
for i,j in enumerate(img_index):
  run_3.append(j + ' ' +test_results[i])

In [0]:
print(*run_3, sep='\n')

0.jpg TallBuilding
1.jpg Forest
2.jpg OpenCountry
3.jpg Mountain
4.jpg store
5.jpg bedroom
6.jpg Office
7.jpg bedroom
8.jpg Forest
9.jpg OpenCountry
10.jpg TallBuilding
11.jpg Forest
12.jpg livingroom
13.jpg Street
14.jpg kitchen
15.jpg TallBuilding
16.jpg Mountain
17.jpg Street
18.jpg industrial
19.jpg OpenCountry
20.jpg OpenCountry
21.jpg Office
22.jpg Office
23.jpg Mountain
24.jpg Highway
25.jpg Insidecity
26.jpg Coast
27.jpg Forest
28.jpg Street
29.jpg OpenCountry
30.jpg OpenCountry
31.jpg store
32.jpg kitchen
33.jpg bedroom
34.jpg OpenCountry
35.jpg store
36.jpg Coast
37.jpg Forest
38.jpg industrial
39.jpg OpenCountry
40.jpg industrial
41.jpg OpenCountry
42.jpg bedroom
43.jpg Office
44.jpg OpenCountry
45.jpg Coast
46.jpg Highway
47.jpg Office
48.jpg store
49.jpg Coast
50.jpg Highway
51.jpg store
52.jpg industrial
53.jpg Mountain
54.jpg Street
55.jpg Suburb
56.jpg TallBuilding
57.jpg TallBuilding
58.jpg Highway
59.jpg Street
60.jpg livingroom
61.jpg Coast
62.jpg OpenCountry
63.jpg 

### Code to train VGG architecture on places365 data. Taken from: https://github.com/iamgroot42/places365_keras

In [0]:
# -*- coding: utf-8 -*-
'''VGG16-places365 model for Keras
# Reference:
- [Places: A 10 million Image Database for Scene Recognition](http://places2.csail.mit.edu/PAMI_places.pdf)
'''

from __future__ import division, print_function
import os

import warnings
import numpy as np

from keras import backend as K
from keras.layers import Input
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers.core import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import _obtain_input_shape

from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16_Places365(include_top=True, weights='places',
                    input_tensor=None, input_shape=None,
                    pooling=None,
                    classes=365):
    """Instantiates the VGG16-places365 architecture.
    Optionally loads weights pre-trained
    on Places. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
                 'places' (pre-training on Places),
                 or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`, or invalid input shape
        """
    if not (weights in {'places', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `places` '
                         '(pre-training on Places), '
                         'or the path to the weights file to be loaded.')

    if weights == 'places' and include_top and classes != 365:
        raise ValueError('If using `weights` as places with `include_top`'
                         ' as true, `classes` should be 365')


    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten =include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5, name='drop_fc1')(x)

        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='drop_fc2')(x)
        
        x = Dense(365, activation='softmax', name="predictions")(x)

    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = Model(inputs, x, name='vgg16-places365')

    # load weights
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

    elif weights is not None:
        model.load_weights(weights)

    return model

  
################################## PART WHERE YOU TEST ABOVE MODEL ON A SPECIMEN IMAGE ############################################################

if __name__ == '__main__':
    #import urllib2
    from urllib.request import urlopen
    import numpy as np
    from PIL import Image
    from cv2 import resize

    TEST_IMAGE_URL = 'http://places2.csail.mit.edu/imgs/demo/6.jpg'

    image = Image.open(urlopen(TEST_IMAGE_URL))
    image = np.array(image, dtype=np.uint8)
    image = resize(image, (224, 224))
    image = np.expand_dims(image, 0)
    print(image.shape)
    model = VGG16_Places365(weights='places')
    predictions_to_return = 5
    preds = model.predict(image)[0]
    print(preds.shape)
    print(type(preds))
    top_preds = np.argsort(preds)[::-1][0:predictions_to_return]
    print(top_preds.shape)
    print(type(top_preds))
    # load the class label
    file_name = 'categories_places365.txt'
    if not os.access(file_name, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    print('--PREDICTED SCENE CATEGORIES:')
    # output the prediction
    for i in range(0, 5):
        print(classes[top_preds[i]])

    # --PREDICTED SCENE CATEGORIES:
    # cafeteria
    # food_court
    # restaurant_patio
    # banquet_hall
    # restaurant

(1, 224, 224, 3)
543088640/543085444 [==============================] - 7s 0us/step
(365,)
<class 'numpy.ndarray'>
(5,)
<class 'numpy.ndarray'>
--PREDICTED SCENE CATEGORIES:
cafeteria
food_court
restaurant_patio
banquet_hall
restaurant
